# Projet IA Frameworks 2023 - Partie 3
@nestorhabibi @julien-blanchon @XuanMinhVuongNGUYEN

## Partie 0 : Librairies, Données

In [1]:
# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms

# data
import pandas as pd
import numpy as np

# random
import random

# os
import os

In [2]:
DEFAULT_RANDOM_SEED = 2021

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
# torch random seed
import torch
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
# basic + tensorflow + torch 
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)
    print('Everything seeded.')

seedEverything()

Everything seeded.


In [3]:
# chargement des données
interactions_test = pd.read_csv('data/interactions_test.csv')
interactions_train = pd.read_csv('data/interactions_train.csv')
RAW_interactions = pd.read_csv('data/RAW_interactions.csv')
RAW_recipes = pd.read_csv('data/RAW_recipes.csv')

In [4]:
# get user id and ratings
df_interactions_train = interactions_train[['user_id', 'recipe_id', 'rating']].to_numpy()
df_interactions_test = interactions_test[['user_id', 'recipe_id', 'rating']].to_numpy()

In [5]:
# split train into train and validation
df_interactions_train, df_interactions_val = torch.utils.data.random_split(df_interactions_train, [0.80, 0.20])

In [6]:
# create dataloaders
trainloader = torch.utils.data.DataLoader(
    df_interactions_train, 
    batch_size=512, 
    shuffle=True, 
    num_workers=2)

valloader = torch.utils.data.DataLoader(
    df_interactions_val, 
    batch_size=512, 
    num_workers=2)

testloader = torch.utils.data.DataLoader(
    df_interactions_test, 
    batch_size=64, 
    num_workers=2)

## Partie 1 : Import de la classe NCF

In [7]:
class NCF(nn.Module):

    def __init__(self, n_users: torch.tensor, n_items: torch.tensor, n_factors: int=8) -> None:
        super().__init__()
        # Embedding layers
        self.user_embeddings = torch.nn.Embedding(n_users, n_factors)
        self.item_embeddings = torch.nn.Embedding(n_items, n_factors)

        # MLP layers
        self.predictor = torch.nn.Sequential(
            nn.Linear(in_features=n_factors*2 , out_features=64),
            nn.Linear(in_features=64, out_features=32),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )


    def forward(self, user: torch.tensor, item: torch.tensor) -> torch.Tensor:
        # Pass through embedding layers
        user_emb = self.user_embeddings(user)
        item_emb = self.item_embeddings(item)

        # Concat the two embeddings
        z = torch.cat([user_emb, item_emb], dim=-1)

        # Pass through MLP
        y = self.predictor(z)
        return y

## Partie 2 : Entraînement de NCF sur les données

In [8]:
# Set device
if ((int(torch.__version__.split(".")[0]) >= 2) or (int(torch.__version__.split(".")[1]) >= 13)) and torch.has_mps:
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


In [9]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from statistics import mean

def train(
        model: NCF, 
        optimizer: torch.optim.Optimizer, 
        trainloader: torch.utils.data.DataLoader, 
        valloader: torch.utils.data.DataLoader,
        epochs: int = 30) -> None:
    criterion_train = nn.MSELoss().to(device)
    criterion_val = nn.L1Loss(reduction='sum').to(device)

    for epoch in range(epochs):
        model.train()
        # initialize progress bar
        pbar = tqdm(trainloader, total=len(trainloader), unit="batch", desc=f"Epoch {epoch: >5}")
        
        # initialize metrics
        train_loss = []

        for i, data in enumerate(pbar):
            # get the data
            users = data[:, 0].to(torch.int).to(device)
            items = data[:, 1].to(torch.int).to(device)
            r = data[:, 2].to(torch.int).to(device)

            # normalize the ratings
            r = (r / 5)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward pass
            y_hat = model(users, items)

            # compute loss
            loss = criterion_train(y_hat.flatten(), r)

            # backward pass + optimize
            loss.backward()
            optimizer.step()

            # update metrics
            train_loss.append(loss.item())

            # update progress bar
            pbar.set_postfix({
                "MSE train": loss.item(),
            })

        pbar.close()

        # Evaluate the model on the val set
        model.eval() 

        with torch.no_grad():
            pbar = tqdm(valloader, total=len(valloader), unit="batch", desc=f"Val")
            for i, data in enumerate(pbar):
                # get the data
                users = data[:, 0].to(torch.int).to(device)
                items = data[:, 1].to(torch.int).to(device)
                r = data[:, 2].to(torch.int).to(device)

                # normalize the ratings
                r = (r / 5)

                y_hat = model(users, items)

                # compute loss
                loss_val = criterion_val(y_hat.flatten(), r)
            
                # update pbar
                pbar.set_postfix({
                    "MAE val": loss_val.item(),
                })

        pbar.close()

In [23]:
# get number of unique user ids and ratings
n_user = interactions_train['user_id'].nunique()
n_items = interactions_train['recipe_id'].nunique()

# to torch
n_user = torch.tensor(n_user).to(device)
n_items = torch.tensor(n_items).to(device)

# define model
model = NCF(n_user, n_items).to(device)

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [24]:
# train the model
train(
    model=model, 
    optimizer=optimizer, 
    trainloader=trainloader, 
    valloader=valloader,
    epochs=10)

Epoch     0:   0%|          | 0/1093 [00:00<?, ?batch/s]

Val:   0%|          | 0/274 [00:00<?, ?batch/s]

Epoch     1:   0%|          | 0/1093 [00:00<?, ?batch/s]

Val:   0%|          | 0/274 [00:00<?, ?batch/s]

Epoch     2:   0%|          | 0/1093 [00:00<?, ?batch/s]

...

Val:   0%|          | 0/274 [00:00<?, ?batch/s]

Epoch     8:   0%|          | 0/1093 [00:00<?, ?batch/s]

Val:   0%|          | 0/274 [00:00<?, ?batch/s]

Epoch     9:   0%|          | 0/1093 [00:00<?, ?batch/s]

Val:   0%|          | 0/274 [00:00<?, ?batch/s]

In [25]:
import math

def test(model, testloader):
    model.eval()
    running_mae = 0
    with torch.no_grad():
        corrects = 0
        total = 0
        pbar = tqdm(testloader, total=len(testloader), unit="batch", desc=f"Test")
        for i, data in enumerate(pbar):
            # get the data
            users = data[:, 0].to(torch.int).to(device)
            items = data[:, 1].to(torch.int).to(device)
            r = data[:, 2].to(torch.int).to(device)

            r = (r / 5)
            y_hat = model(users, items).flatten()
            error = torch.abs(y_hat - r).sum().data
            
            running_mae += error
            total += r.size(0)
    
    mae = running_mae/total
    return (mae * 5).item()

In [26]:
test(model, testloader)

Test:   0%|          | 0/195 [00:00<?, ?batch/s]

0.874189019203186

In [27]:
# save weights of the model
torch.save(model.state_dict(), 'weights.pth')